<a href="https://colab.research.google.com/github/DinhVinh2404/AI_Agent/blob/main/AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install python-dotenv
%pip install langgraph
%pip install langchain-openai
%pip install langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.5/447.5 kB 30.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.75
    Uninstalling langchain-core-0.3.75:
      Successfully uninstalled langchain-core-0.3.75
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall:

In [6]:
!pip install -U langchain-tavily

In [27]:
# !pip install -U langchain-together
!pip install -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 68.2 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.7.0 which is incompatible.


In [29]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_tavily import TavilySearch

In [28]:
from dotenv import load_dotenv
_ = load_dotenv()

In [ ]:
from google.colab import userdata
userdata.get('OPENAI_API_KEY')

**SET TOOLS**

In [9]:
tavily_api_key = userdata.get('TAVILY_API_KEY')
tool = TavilySearch(max_results=4, tavily_api_key=tavily_api_key)

In [10]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [42]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [58]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash", api_key= userdata.get('GOOGLE_API_KEY'))
abot = Agent(model, [tool], system=prompt)
from IPython.display import Image
# Image(abot.graph.get_graph().draw_png())

**QUERY**

In [69]:
query = "Lịch duyệt binh của Việt Nam gần nhất? Có bao nhiêu khối tham gia?"
messages = [HumanMessage(content=query)]
result = abot.graph.invoke({"messages": messages})


Calling: {'name': 'tavily_search', 'args': {'query': 'Lịch duyệt binh của Việt Nam gần nhất? Có bao nhiêu khối tham gia?'}, 'id': '4bab2ba2-6fb3-4e04-b978-9fff4e5e9bec', 'type': 'tool_call'}
Back to the model!


In [70]:
result

{'messages': [HumanMessage(content='Lịch duyệt binh của Việt Nam gần nhất? Có bao nhiêu khối tham gia?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search', 'arguments': '{"query": "L\\u1ecbch duy\\u1ec7t binh c\\u1ee7a Vi\\u1ec7t Nam g\\u1ea7n nh\\u1ea5t? C\\u00f3 bao nhi\\u00eau kh\\u1ed1i tham gia?"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--7a11d347-e945-411f-a524-b560e8d68827-0', tool_calls=[{'name': 'tavily_search', 'args': {'query': 'Lịch duyệt binh của Việt Nam gần nhất? Có bao nhiêu khối tham gia?'}, 'id': '4bab2ba2-6fb3-4e04-b978-9fff4e5e9bec', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1593, 'output_tokens': 111, 'total_tokens': 1704, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 78}}),
  ToolMessage(content="{'query': '

In [71]:
print(result['messages'][-1].content)

Việt Nam sẽ tổ chức lễ diễu binh, duyệt binh quy mô lớn mang tên A80 vào ngày 2/9/2025 để kỷ niệm 80 năm Quốc khánh.

Theo thông tin sơ bộ, lễ diễu binh, duyệt binh A80 sẽ có sự tham gia của 38 khối diễu binh và diễu hành, bao gồm 11 khối đứng và 27 khối đi. Một số nguồn khác cũng đưa ra con số lên đến hơn 16.000 người tham gia với 43 khối đi, 18 khối đứng cùng 14 khối xe pháo quân sự, đặc chủng của Quân đội, Công an.
